In [ ]:
import os
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import math

/home/zhaochuyang/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
PACS_DIR = "/home/chuyang/Workspace/data/PACS"

In [ ]:
image_paths = []

for root, dirs, files in os.walk(PACS_DIR):
    for filename in files:
        path = os.path.join(root, filename)
        if path.lower().endswith(("jpg", "png", "jpeg", "bmp")):
            image_paths.append(path)

print("Total images: {}".format(len(image_paths)))

: 

In [ ]:
def extract(a, t, x_shape):
    b, *_ = t.shape
    out = a.gather(-1, t)
    return out.reshape(b, *((1,) * (len(x_shape) - 1)))


def cosine_beta_schedule(timesteps, s = 0.008):
    """
    cosine schedule
    as proposed in https://openreview.net/forum?id=-NEXDKk8gZ
    """
    steps = timesteps + 1
    x = torch.linspace(0, timesteps, steps, dtype = torch.float64)
    alphas_cumprod = torch.cos(((x / timesteps) + s) / (1 + s) * math.pi * 0.5) ** 2
    alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
    betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])
    return torch.clip(betas, 0, 0.999)


def linear_beta_schedule(timesteps):
    scale = 1000 / timesteps
    beta_start = scale * 0.0001
    beta_end = scale * 0.02
    return torch.linspace(beta_start, beta_end, timesteps, dtype = torch.float64)


betas = linear_beta_schedule(timesteps)
alphas = 1. - betas
alphas_cumprod = torch.cumprod(alphas, dim=0)
sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1. - alphas_cumprod)


def add_noise(image, n_times):
    n_times = torch.as_tensor([n_times])
    timesteps = 1000
    noise = torch.randn_like(image)
    noisy_image = extract(sqrt_alphas_cumprod, n_times, image.shape) * image + \
            extract(sqrt_one_minus_alphas_cumprod, n_times, image.shape) * noise
    return noisy_image


def convert_tensor_to_image(input):
    image = input.cpu().numpy()
    # image = np.transpose(image, (1, 2, 0))
    # image = (image - image.min()) / (image.max() - image.min())
    image = np.uint8(np.clip(image * 255, 0, 255))
    return image

In [ ]:
N = 4
random.shuffle(image_paths)
for i in range(N):
    image = cv2.imread(image_paths[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # print(image.shape)
    plt.figure()
    plt.imshow(image)

    # z-score normalization
    mean = np.mean(image, axis=(0, 1))
    std = np.std(image, axis=(0, 1))
    image = (image - mean) / std

    image = add_noise(image, 100)
    image = convert_tensor_to_image(image)

    # unnormalize to range 0-1
    image = (image - image.min()) / (image.max() - image.min())
    plt.figure()
    plt.imshow(image)
    